# Ultranest for Nested Sampling

We will look at the following data to investigate nested sampling using the `ultranest` package. 

In [ ]:
import pandas as pd

data = pd.read_csv('../data/signals.csv')
data.head()

This data is the measurement of some signal $y$, as a function of $x$. 

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.errorbar(data['x'], data['y'], yerr=data['yerr'])
ax.set_xlabel('x')
ax.set_ylabel('y')
plt.show()

The question we want to answer about this data is if one or two Gaussian signals are present. 
We can think of this as two models: one for a single signal and a more complex one for two signals. 
This is the perfect opportunity to use Bayesian model selection. 

Before constructing either model, we will create the likelihood function, which will be the same either way but with a different input model. 

In [3]:
import numpy as np
from scipy.stats import norm

data_distribution = [norm(loc=loc, scale=scale) for loc, scale in zip(data['y'], data['yerr'])]

def likelihood(params, model):
    """
    A general likelihood function for a model with Gaussian errors.
    
    :param params: The parameters of the model.
    :param model: The model function.
    
    :return: The likelihood of the model given the data.
    """
    model_y = model(data['x'], params)
    return np.sum([d.logpdf(m) for d, m in zip(data_distribution, model_y)])

The next step is to build a simpler model and compute the evidence. 
We start by creating priors for the mean and standard deviation of the single Gaussian function. 

In [4]:
from scipy.stats import uniform, norm

priors_one = [norm(220, 20), 
              uniform(10, 20)]

The mean has a normally distributed prior ($\mathcal{N}(220, 20)$) and the standard deviation is uniform ($\mathcal{U}(10, 20)$).

The next step is to create a prior transform function. 
This function converts uniformly distributed random variables to random variables drawn from the priors of interest. 

In [5]:
def prior_transform_one(u):
    """
    Transform the uniform random variables `u` to the model parameters.
    
    :param u: Uniform random variables
    
    :return: Model parameters
    """
    return [p.ppf(u_) for p, u_ in zip(priors_one, u)]

Now, we construct the model and create a model-specific likelihood and function to be fed to the sampler. 

In [6]:
def model_one(x, params):
    """
    A simpler single Gaussian model.
    
    :param x: The x values
    :param params: The model parameters
    
    :return: The y values
    """
    return norm(loc=params[0], scale=params[1]).pdf(x)

def likelihood_one(params):
    """
    The likelihood function for the simpler model.
    
    :param params: The model parameters
    
    :return: The likelihood
    """
    return likelihood(params, model_one)

This is then passed to the `ultranest.ReactiveNestedSampler`. 

In [ ]:
import ultranest

sampler_one = ultranest.ReactiveNestedSampler(['mu1', 'sigma1'], likelihood_one, prior_transform_one)
sampler_one.run(show_status=False)
sampler_one.print_results()

In [ ]:
sampler_one.results['logz'], sampler_one.results['logzerr']

We can now do the same process for a more complex two-Gaussian model. 
Note that we have four priors now instead of two. 

In [ ]:
priors_two = [norm(220, 20), 
              uniform(10, 20), 
              norm(500, 100),
              uniform(100, 200)]

def prior_transform_two(u):
    """
    Transform the uniform random variables `u` to the model parameters for the more complex model.
    
    :param u: Uniform random variables
    
    :return: Model parameters
    """
    return [p.ppf(u_) for p, u_ in zip(priors_two, u)]

def model_two(x, params):
    """
    A more complex double Gaussian model.
    
    :param x: The x values
    :param params: The model parameters
    
    :return: The y values
    """
    return (norm(loc=params[0], scale=params[1]).pdf(x) + norm(loc=params[2], scale=params[3]).pdf(x)) * 0.5

def likelihood_two(params):
    """
    The likelihood function for the more complex model.
    
    :param params: The model parameters
    
    :return: The likelihood
    """
    return likelihood(params, model_two)

sampler_two = ultranest.ReactiveNestedSampler(['mu1', 'sigma1', 'mu2', 'sigma2'], likelihood_two, prior_transform_two)
sampler_two.run(show_status=False)
sampler_two.print_results()

In [ ]:
sampler_two.results['logz'], sampler_two.results['logzerr']

We can now calculate the Bayes factor for these two models. 

In [ ]:
log_B = sampler_two.results['logz'] - sampler_one.results['logz']
log_B

There is decisive evidence for the more complex model. 
Therefore, we would be able to continue our analysis by considering the signal shown by the two Gaussians. 